In [6]:
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


crime = pd.read_csv('05. crime_dataset_analysis.csv')

X = crime.drop(columns=["Weapon Category"])
y = crime["Weapon Category"]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)





In [7]:
# XGBoost
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

scale_pos_weight = len(y_train) / sum(y_train == 1)  # Number of negative to positive samples

# Initialize XGBoost model with scale_pos_weight
xgb_model = XGBClassifier(random_state=42,
                          eval_metric='aucpr',
                          scale_pos_weight=scale_pos_weight)

# Fit the model on the training data
xgb_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = xgb_model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Display the classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.7031093933208842
              precision    recall  f1-score   support

           0       0.73      0.16      0.26     15157
           1       0.70      0.97      0.81     30897

    accuracy                           0.70     46054
   macro avg       0.71      0.56      0.54     46054
weighted avg       0.71      0.70      0.63     46054



In [4]:
confusion_matrix(y_test, y_pred)

array([[ 2387, 12770],
       [  903, 29994]])

In [8]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"Original class distribution:\n{y_train.value_counts()}")
print(f"Resampled class distribution:\n{y_train_resampled.value_counts()}")

# Step 3: Train the XGBoost classifier
xgb_model.fit(X_train_resampled, y_train_resampled)

# Step 4: Evaluate on the test set
y_pred = xgb_model.predict(X_test)
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Display the classification report
print(classification_report(y_test, y_pred))


Original class distribution:
Weapon Category
1    72091
0    35366
Name: count, dtype: int64
Resampled class distribution:
Weapon Category
1    72091
0    72091
Name: count, dtype: int64
Accuracy: 0.6983758196899292
              precision    recall  f1-score   support

           0       0.57      0.33      0.42     15157
           1       0.73      0.88      0.80     30897

    accuracy                           0.70     46054
   macro avg       0.65      0.60      0.61     46054
weighted avg       0.68      0.70      0.67     46054



In [7]:
confusion_matrix(y_test, y_pred)

array([[ 4939, 10218],
       [ 3673, 27224]])